## Kaggle Steel Defects - Merged Classification and Segmentation

Link to competition: https://www.kaggle.com/c/severstal-steel-defect-detection

This notebook was converted from my prior Kaggle notebook.  Migrated to TF 2.x and converted various methods to be more native TF.  This notebook combines the Classification and Segmentation models.  A first classification pass is done on all images.  Images that were identified as having defects were then use to on the second segmentation pass.  The final step is to merge the results into a simgle dataframe.  The final dice_coef score is from using the training images and comparing with the actual ground truth.


Final dice_coef Score from Training images:  0.8544026199594487

In [ ]:
#"""
# Google Collab specific stuff....
from google.colab import drive
drive.mount('/content/drive')

import os
!ls "/content/drive/My Drive"

USING_COLLAB = True
%tensorflow_version 2.x
#"""

In [ ]:
# To start, install kaggle libs
#!pip install -q kaggle

# Workaround to install the newest version
# https://stackoverflow.com/questions/58643979/google-colaboratory-use-kaggle-server-version-1-5-6-client-version-1-5-4-fai
!pip install kaggle --upgrade --force-reinstall --no-deps

In [ ]:
# Upload your "kaggle.json" file that you created from your Kaggle Account tab
# If you downloaded it, it would be in your "Downloads" directory

from google.colab import files
files.upload()

In [ ]:
# On your VM, create kaggle directory and modify access rights

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
#!kaggle competitions list
!kaggle competitions download -c severstal-steel-defect-detection

In [ ]:
!unzip -uq severstal-steel-defect-detection.zip 
!ls train_images/a75bb4c01*.*

In [ ]:
# Cleanup to add some space....
!rm -r test_images
!rm severstal-steel-defect-detection.zip

In [ ]:
# Setup sys.path to find MachineLearning lib directory

try: USING_COLLAB
except NameError: USING_COLLAB = False

%load_ext autoreload
%autoreload 2

import sys
if "MachineLearning" in sys.path[0]:
    pass
else:
    print(sys.path)
    if USING_COLLAB:
        sys.path.insert(0, '/content/drive/My Drive/GitHub/MachineLearning/lib')  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    else:
        sys.path.insert(0, '/Users/john/Documents/GitHub/MachineLearning/lib')  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    
    print(sys.path)

In [ ]:
#%reload_ext autoreload


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os, sys, random, warnings, time, copy, csv, gc
import numpy as np 

import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from tqdm import tqdm_notebook, tnrange, tqdm
import pandas as pd

import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.models import load_model 

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

AUTOTUNE = tf.data.experimental.AUTOTUNE
print("AUTOTUNE: ", AUTOTUNE)

from TrainingUtils import *
from losses_and_metrics.Losses_Babakhin import make_loss, Kaggle_IoU_Precision, dice_coef_loss_bce

#warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)

## Helper methods


In [ ]:
# GLOBALS/CONFIG ITEMS

# Set root directory path to data
if USING_COLLAB:
    #ROOT_PATH = "/content/drive/My Drive/ImageData/KaggleSteelDefects"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    ROOT_PATH = ""
    MODEL_PATH= "/content/drive/My Drive/ImageData/KaggleSteelDefects"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    
else:
    ROOT_PATH = "/Users/john/Documents/ImageData/KaggleSteelDefects"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    MODEL_PATH= "/Users/john/Documents/ImageData/KaggleSteelDefects"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    
# Establish global Classification dictionary
parms_class = GlobalParms(MODEL_NAME="model-SteelDefects-Classification-V01.h5",
                    ROOT_PATH=ROOT_PATH,
                    TEST_PATH="train_images", 
                    MODEL_PATH=MODEL_PATH,
                    NUM_CLASSES=2,
                    CLASS_NAMES=["Good", "Defect"],
                    IMAGE_ROWS=224,
                    IMAGE_COLS=224,
                    IMAGE_CHANNELS=3,
                    BATCH_SIZE=32)

parms_class.print_contents()

# Establish global Segmentation dictionary
parms_seg = GlobalParms(MODEL_NAME="model-SteelDefects-Segmentation-V01.h5",
                    ROOT_PATH=ROOT_PATH,
                    TEST_PATH="train_images", 
                    MODEL_PATH=MODEL_PATH,
                    NUM_CLASSES=4,
                    CLASS_NAMES=["1", "2", "3", "4"],
                    IMAGE_ROWS=256,
                    IMAGE_COLS=800,
                    IMAGE_CHANNELS=3,
                    BATCH_SIZE=32)


parms_seg.print_contents()

# Other globals...
ORIG_MASK_SHAPE = (256, 1600)


In [ ]:
# Simple helper method to display batches of images with labels....  

def show_segmentation_image_masks(image_in, masks_in=None):
    if tf.is_tensor(image_in):
        image = image_in.numpy()
    else:
        image = image_in

    if tf.is_tensor(masks_in): 
        masks = masks_in.numpy()
    else:
        masks = masks_in

    #print(image.shape, masks.shape)

    # cv2.polylines and cv2.findContours display better when range is 0-255
    # https://docs.opencv.org/2.4/modules/core/doc/drawing_functions.html
    image = image * 255
    palet = [(249, 192, 12), (0, 185, 241), (114, 0, 218), (249,50,12)]
    fig, ax = plt.subplots(1,1,figsize=(20,10))
    if masks is not None:
        title = "Labels: "
        for j in range(parms_seg.NUM_CLASSES):
            msk = np.ascontiguousarray(masks[:, :, j], dtype=np.uint8)
            if np.count_nonzero(msk) > 0:
                title = title + str(j+1) + ",  "
                contours, _ = cv2.findContours(msk, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
                for i in range(0, len(contours)):
                    cv2.polylines(image, contours[i], True, palet[j], 2) 

        title = title[:-3]  
        ax.set_title(title)

    #ax.imshow(tf.keras.preprocessing.image.array_to_img(image), cmap=plt.get_cmap('gray'))
    #print(image.shape, image.dtype, np.max(image), np.min(image))
    ax.imshow(image/255, cmap=plt.get_cmap('gray'))


def show_segmentation_batch_image_masks(image, masks=None):
    for i in range(len(image)):
        if masks is None:
            show_segmentation_image_masks(image[i])
        else:
            show_segmentation_image_masks(image[i], masks[i])


# Simple helper method to display batches of clasification images      
def show_classification_batch(image_batch, label_batch=None, number_to_show=25, r=5, c=5, print_shape=False):
    show_number = min(number_to_show, parms_class.BATCH_SIZE)
    show_number = min(show_number, len(image_batch))

    if show_number < 8: #if small number, then change row, col and figure size
        if parms_class.IMAGE_COLS > 64 or parms_class.IMAGE_ROWS > 64:
            plt.figure(figsize=(25,25)) 
        else:
            plt.figure(figsize=(10,10))  
        r = 4
        c = 2 
    else:
        plt.figure(figsize=(10,10))  

    for n in range(show_number):
        if print_shape:
            print("Image shape: {}  Max: {}  Min: {}".format(image_batch[n].shape, 
                                                             np.max(image_batch[n]), 
                                                             np.min(image_batch[n])))
        ax = plt.subplot(r,c,n+1)
        cmap="gray"
        if len(image_batch[n].shape) == 3:
            if image_batch[n].shape[2] == 3:
                cmap="viridis"
        plt.imshow(tf.keras.preprocessing.image.array_to_img(image_batch[n]), cmap=plt.get_cmap(cmap))
        if label_batch is not None:
            plt.title(parms.CLASS_NAMES[np.argmax(label_batch[n])])

        plt.axis('off')

In [ ]:
# Helper methods to create mask's or rle's
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle2mask(rle, input_shape):
    width, height = input_shape[:2]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return mask.reshape(height, width).T

def build_masks(rles, input_shape):
    depth = len(rles)
    masks = np.zeros((*input_shape, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            masks[:, :, i] = rle2mask(rle, input_shape)
    
    return masks

def build_rles(masks):
    width, height, depth = masks.shape
    
    rles = [mask2rle(masks[:, :, i]) for i in range(depth)]
    
    return rles


In [ ]:

from tensorflow.keras.models import load_model
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta, Adam, Nadam, SGD
########
K = tf.keras.backend

loss_function = "bce_dice"  # bce_dice, lovasz
loss = make_loss(loss_function)    

def dice_coef_np(y_true, y_pred, smooth=1):
    y_true_f = np.ndarray.flatten(y_true)
    y_pred_f = np.ndarray.flatten(y_pred)
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)



## Classification

In [ ]:
# Get all file names
image_file_list = load_file_names_Util(parms_class.TEST_PATH,
                                       parms_class.IMAGE_EXT,
                                       full_file_path=False)
print(image_file_list[:5])

# Set steps and Create train ALL csv
all_df = pd.DataFrame(image_file_list, columns=["ImageId"])
all_df.head()

In [ ]:
# Mapped method to load classification images
def process_load_calssification_image(image_id: tf.Tensor) -> tf.Tensor:
    file_path = parms_class.TEST_PATH + "/" + image_id
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=parms_class.IMAGE_CHANNELS)
    image = tf.image.convert_image_dtype(image, parms_class.IMAGE_DTYPE)
    image = tf.image.resize(image, [parms_class.IMAGE_ROWS, parms_class.IMAGE_COLS])

    return image_id, image

In [ ]:
# Create Dataset from pf
all_dataset = tf.data.Dataset.from_tensor_slices(all_df["ImageId"].values)
                                               
# Verify image and label were loaded
for image_id in all_dataset.take(2):
    print("Image ID: ", image_id.numpy().decode("utf-8"))

# map training images to processing, includes any augmentation
all_dataset = all_dataset.map(process_load_calssification_image, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image_id, image in all_dataset.take(1):
    print("ImageId: ", image_id.numpy().decode("utf-8"))
    print("Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    some_image = image.numpy()

all_dataset = all_dataset.prefetch(1).repeat()

# Uncomment to show the batch of images, execute this cell multiple times to see the images
#for batch_image in train_dataset.take(1):
#    show_classification_batch(batch_image)

show_classification_batch([some_image])

In [ ]:
#Load saved model
model_class = load_model(parms_class.MODEL_PATH)
print("Loaded model: ", parms_class.MODEL_PATH)

In [ ]:
# Use model to generate predicted labels and probabilities

def create_classification_predictions(model_actual,
                              dataset,
                              steps):
    """
      Uses dataset to predict results and return list.

      Args:
        model_actual : trained model to use for predictions
        dataset : dataset iterator
        steps : number of batches to process
        batch_size : size of the batch

      Returns:
        pred_results :  list of the ImageId's and classification result

    """

    classification_list = []
    no_defect_list = []
    defect_list = []

    for batch_image_id, batch_image in tqdm(dataset.take(steps)):
        image_id = batch_image_id.numpy().decode("utf-8")
        image = batch_image

        image = np.expand_dims(image, axis=0)
        predict_probabilities_tmp = model_actual.predict(image)[0]
        predict_label = np.argmax(predict_probabilities_tmp)

        classification_list.append([image_id, predict_label, predict_probabilities_tmp])
        if predict_label == 0:
            no_defect_list.append(image_id)
        else:
            defect_list.append(image_id)

    return classification_list, no_defect_list, defect_list


In [ ]:
classification_list, no_defect_list, defect_list = create_classification_predictions(model_class, all_dataset, len(all_df))
#classification_list, no_defect_list, defect_list = create_classification_predictions(model_class, all_dataset, 10)
print(len(classification_list), classification_list[:10])
print(len(no_defect_list), no_defect_list[:4])
print(len(defect_list), defect_list[:4])

In [ ]:
classification_results = []
for i, image_id in enumerate(no_defect_list):
    classification_results.append([image_id+"_1", ""])
    classification_results.append([image_id+"_2", ""])
    classification_results.append([image_id+"_3", ""])
    classification_results.append([image_id+"_4", ""])

print(classification_results[:6])

## Segmentation

In [ ]:
#Load saved model
model_seg = load_model(parms_seg.MODEL_PATH, custom_objects={'loss': loss, 'dice_coef': dice_coef})
print("loaded: ", parms_seg.MODEL_PATH)

In [ ]:
# build defect_df
defect_df = pd.DataFrame(defect_list, columns=["ImageId"])
defect_df.head()

In [ ]:
# Mapped method to load segmentation images
def process_load_segmentation_image(image_id: tf.Tensor) -> tf.Tensor:
    file_path = parms_seg.TEST_PATH + "/" + image_id
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=parms_seg.IMAGE_CHANNELS)
    image = tf.image.convert_image_dtype(image, parms_seg.IMAGE_DTYPE)
    image = tf.image.resize(image, [parms_seg.IMAGE_ROWS, parms_seg.IMAGE_COLS])

    return image_id, image


In [ ]:
# Create Dataset from pf
defect_dataset = tf.data.Dataset.from_tensor_slices(defect_df["ImageId"].values)
                                               
# Verify image and label were loaded
for image_id in defect_dataset.take(2):
    print("Image ID: ", image_id.numpy().decode("utf-8"))

# map training images to processing, includes any augmentation
defect_dataset = defect_dataset.map(process_load_segmentation_image, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image_id, image in defect_dataset.take(1):
    print("ImageId: ", image_id.numpy().decode("utf-8"))
    print("Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    some_image = image.numpy()

defect_dataset = defect_dataset.prefetch(1).repeat()

# Uncomment to show the batch of images, execute this cell multiple times to see the images
#for batch_image in defect_dataset.take(1):
#    show_segmentation_batch_image_masks(batch_image)

show_segmentation_batch_image_masks([some_image])

In [ ]:
def predictions_using_dataset_masks(model_actual,
                              dataset,
                              steps,
                              pred_threshold=0.50):
    """
      Uses generator to predict results.  Builds actual_labels, predict_labels
      and predict_probabilities

      Args:
        model_actual : trained model to use for predictions
        dataset : dataset iterator
        steps : number of batches to process

      Returns:
        list of predicted defects with class and rle

    """

    segmentation_results = []

    for batch_image_id, batch_image in tqdm(dataset.take(steps)):

        image_id = batch_image_id.numpy().decode("utf-8")
        image = batch_image

        image = np.expand_dims(image, axis=0)
        # image = tf.reshape(image, (1, *image.shape))

        predict_probabilities_tmp = model_actual.predict(image)[0]

        pred_masks = np.where(predict_probabilities_tmp > pred_threshold, 1, 0)
        if np.count_nonzero(pred_masks) == 0:
            #print("none predicted found, adjusting np.where....")
            pred_masks = np.where(predict_probabilities_tmp > pred_threshold / 2, 1, 0) 

        #print(image_id, "  pred_masks ", np.max(pred_masks), np.min(pred_masks))

        pred_masks = np.resize(pred_masks, (*ORIG_MASK_SHAPE, parms_seg.NUM_CLASSES)) 
        pred_masks = np.where(pred_masks > 0.5, 1, 0)

        pred_rles = build_rles(pred_masks)
        #print(image_id, "  pred_rles ", pred_rles)
        segmentation_results.append([image_id+"_1", pred_rles[0]])
        segmentation_results.append([image_id+"_2", pred_rles[1]])
        segmentation_results.append([image_id+"_3", pred_rles[2]])
        segmentation_results.append([image_id+"_4", pred_rles[3]])

    return segmentation_results

In [ ]:
# Use model to generate predicted labels and probabilities

segmentation_results = predictions_using_dataset_masks(model_seg, defect_dataset, len(defect_df))
#segmentation_results = predictions_using_dataset_masks(model_seg, defect_dataset, 10)
print(segmentation_results[:4])

## Create submission file

In [ ]:
#Create a df from both lists, 004f40c73.jpg_1, ImageId_ClassId,EncodedPixels
submission_list = segmentation_results + classification_results
sorted(submission_list)
submission_df = pd.DataFrame(submission_list, columns =["ImageId_ClassId", "EncodedPixels"])

#submission_df.to_csv("<your directory>/submission.csv")

submission_df.head()

## Validate results against training

In [ ]:
# Load train DEFECT csv
image_defect_df = pd.read_csv(os.path.join(parms_seg.ROOT_PATH, "train.csv"))
image_defect_df["ImageId_ClassId"] = image_defect_df["ImageId"]+"_"+image_defect_df["ClassId"].astype(str)

image_defect_df.head()

In [ ]:
def score_rle(defect_rle, sub_rle):
    # did not find a real entry and no entry in submission, score 1
    #print("rle sub: ", len(sub_rle), type(sub_rle), sub_rle,  "  defect: ", len(defect_rle), type(defect_rle), defect_rle)

    if (defect_rle == "") and (sub_rle == ""):
        #print("Score: ", 1)
        return 1.0
    else: 
      # score rle's using dice
        mask_defect = rle2mask(defect_rle, ORIG_MASK_SHAPE)
        mask_sub = rle2mask(sub_rle, ORIG_MASK_SHAPE)
        #print("mask defect ", mask_defect)
        #print("mask sub ", mask_sub)

        score = dice_coef_np(mask_defect, mask_sub)
        #print("Score: ", score)
        return score

score = 0.0
for i in tqdm(range(len(submission_list))):
    sub_image_id = submission_list[i][0]
    sub_rle = submission_list[i][1]
    #print(sub_image_id, sub_rle)
    image_class_df = image_defect_df.loc[image_defect_df["ImageId_ClassId"] == sub_image_id] 
    tmp_str = image_class_df["EncodedPixels"].values
    if len(tmp_str) == 0:
        tmp_str = ""
    else:
        tmp_str = tmp_str[0]
    #print("asdfg ",len(tmp_str), tmp_str)
    score += score_rle(tmp_str, sub_rle)

print("")
print("Score: ", score / len(submission_list))
